In [1]:
# To install the official MySQL Connector for Python, please use the name mysql-connector-python:
# pip install mysql-connector-python
import mysql.connector
import datetime
import pandas as pd
import numpy as np

In [2]:
import urllib.request, json

In [3]:
df = pd.read_csv('atricle_sampler.csv')
df.doi = df.doi.str.lower()
df.head()

,doi,title,author,year_published,field of study,is_oa
0,10.3842/sigma.2012.016,Introduction to loop quantum cosmology\n,"Banerjee, K., Calcagni, G., & Martín-Benito, M.",2010,Physics,1
1,10.1016/j.physrep.2012.01.001,Modified gravity and cosmology,"Clifton, T., Ferreira, P. G., Padilla, A., & S...",2012,Physics,1
2,10.1146/annurev-astro-081811-125615,Cosmic star-formation history,"Madau, P., & Dickinson, M.",2014,Physics,1
3,10.1016/j.physletb.2008.07.018,Review of Particle Physics\n,"C. Amsler, M. Doser, M. Antonelli, D. M. Asner...",2008,Physics,1
4,10.1126/science.1202043,Improved Learning in a Large-Enrollment Physic...,"Deslauriers, L., Schelew, E., & Wieman, C.",2011,Physics,1


In [4]:
cnx = mysql.connector.connect(user='scott', password='tiger',
                              host='127.0.0.1',
                              database='saa',
                              use_pure=False)
cursor = cnx.cursor()

In [5]:
doi50 ='\',\''.join(df.doi)
doi50

"10.3842/sigma.2012.016','10.1016/j.physrep.2012.01.001','10.1146/annurev-astro-081811-125615','10.1016/j.physletb.2008.07.018','10.1126/science.1202043','10.1140/epjc/s10052-016-4099-4','10.1016/j.aop.2005.04.002','10.1103/physrevlett.94.111601','10.1143/ptps.183.1','10.1143/ptp.113.843','10.1371/journal.pone.0064841','10.1108/jd-03-2016-0030','10.1007/s10900-018-0547-4','10.1016/j.ijinfomgt.2018.07.004','10.1016/j.lisr.2017.03.001','10.1016/j.lisr.2017.03.005','10.1002/asi.23124','10.1145/2998181.2998204','10.1108/jd-06-2017-0095','10.1080/10447318.2017.1365459','10.1177/1464884905056815','10.1177/1464884903004001484','10.1080/1461670x.2011.571825','10.1080/17512780802281065','10.1080/17512786.2012.667269','10.1207/s15506878jobem4604_3','10.1177/1461444809341393','10.1177/0016549205057564','10.1080/14616700500533643','10.1080/14616700118394','10.3390/ijerph14091002','10.1097/phh.0000000000000347','10.11604/pamj.supp.2017.27.1.12570','10.1093/pubmed/fdw130','10.24095/hpcdp.38.7/8.02',

In [6]:
query = ("SELECT doi, is_oa FROM tmp "
        "WHERE doi IN (\'{}\')").format(doi50)
print (query)

SELECT doi, is_oa FROM tmp WHERE doi IN ('10.3842/sigma.2012.016','10.1016/j.physrep.2012.01.001','10.1146/annurev-astro-081811-125615','10.1016/j.physletb.2008.07.018','10.1126/science.1202043','10.1140/epjc/s10052-016-4099-4','10.1016/j.aop.2005.04.002','10.1103/physrevlett.94.111601','10.1143/ptps.183.1','10.1143/ptp.113.843','10.1371/journal.pone.0064841','10.1108/jd-03-2016-0030','10.1007/s10900-018-0547-4','10.1016/j.ijinfomgt.2018.07.004','10.1016/j.lisr.2017.03.001','10.1016/j.lisr.2017.03.005','10.1002/asi.23124','10.1145/2998181.2998204','10.1108/jd-06-2017-0095','10.1080/10447318.2017.1365459','10.1177/1464884905056815','10.1177/1464884903004001484','10.1080/1461670x.2011.571825','10.1080/17512780802281065','10.1080/17512786.2012.667269','10.1207/s15506878jobem4604_3','10.1177/1461444809341393','10.1177/0016549205057564','10.1080/14616700500533643','10.1080/14616700118394','10.3390/ijerph14091002','10.1097/phh.0000000000000347','10.11604/pamj.supp.2017.27.1.12570','10.1093/p

In [7]:
cursor.execute(query)
query_result = {}
for (doi, is_oa) in cursor:
    if is_oa == 'False':
        query_result[doi] = 0
    else:
        query_result[doi] = 1
len(query_result)

42

In [8]:
diff = list(set(df.doi)-set(query_result))
diff

['10.1207/s15506878jobem4604_3',
 '10.1016/j.physletb.2008.07.018',
 '10.1080/17512780802281065',
 '10.2753/mis0742-1222240302',
 '10.1103/physrevlett.94.111601',
 '10.1097/phh.0000000000000347',
 '10.1177/0016549205057564',
 '10.1145/330908.331819']

In [9]:
crossref_base = 'https://api.crossref.org/works/{}/agency'

In [10]:
crsref_resp,crsref_err = [], []
for doi in diff:
    try:
        with urllib.request.urlopen(crossref_base.format(doi)) as url:
            crsref_resp.append(json.loads(url.read().decode()))
    except (UnicodeEncodeError, urllib.error.HTTPError) as e:
        crsref_err.append({doi : 'Code: {c}, Message, {m}'.format(c = type(e).__name__, m = str(e))})
        continue

In [12]:
crsref_err

[]

In [14]:
vaild_doi = [json['message']['DOI'] for json in crsref_resp]
print(len(vaild_doi))

8


In [15]:
upw_base = 'https://api.unpaywall.org/v2/{}?email=YOUR_EMAIL'
upw_resp = []
for doi in vaild_doi:
    with urllib.request.urlopen(upw_base.format(doi)) as url:
        upw_resp.append(json.loads(url.read().decode()))

In [17]:
api_add_result = {json['doi']: 0 if json['is_oa'] != True else 1 for json in upw_resp}
api_add_result

{'10.1207/s15506878jobem4604_3': 0,
 '10.1016/j.physletb.2008.07.018': 0,
 '10.1080/17512780802281065': 1,
 '10.2753/mis0742-1222240302': 1,
 '10.1103/physrevlett.94.111601': 1,
 '10.1097/phh.0000000000000347': 1,
 '10.1177/0016549205057564': 0,
 '10.1145/330908.331819': 0}

In [18]:
df_2 = pd.DataFrame(list({**query_result, **api_add_result}.items()), columns=['doi','upw_oa'])
df_2.upw_oa = df_2.upw_oa.astype(int)
df_2.head()

,doi,upw_oa
0,10.1002/asi.23124,0
1,10.1007/s10900-018-0547-4,0
2,10.1016/j.aop.2005.04.002,1
3,10.1016/j.ejor.2009.08.003,0
4,10.1016/j.epidem.2014.08.008,1


In [19]:
df.is_oa = df.is_oa.astype(int)
df_result = pd.merge(df[['doi', 'field of study', 'is_oa']], df_2,
                     on='doi', how='right')
df_result

,doi,field of study,is_oa,upw_oa
0,10.3842/sigma.2012.016,Physics,1,1
1,10.1016/j.physrep.2012.01.001,Physics,1,1
2,10.1146/annurev-astro-081811-125615,Physics,1,1
3,10.1016/j.physletb.2008.07.018,Physics,1,0
4,10.1126/science.1202043,Physics,1,1
5,10.1140/epjc/s10052-016-4099-4,Physics,1,1
6,10.1016/j.aop.2005.04.002,Physics,1,1
7,10.1103/physrevlett.94.111601,Physics,1,1
8,10.1143/ptps.183.1,Physics,1,1
9,10.1143/ptp.113.843,Physics,1,1


In [ ]:
df_result['is_same'] = df_result.apply(lambda x : 0 if x['is_oa'] != x['upw_oa'] else 1, axis=1)
df_result

In [ ]:
df_result.groupby('is_same').count()

In [ ]:
24/38 

# Below is Running time counts

In [ ]:
print ('Start Time: ' + str(datetime.datetime.now()))
query = ("SELECT doi, is_oa FROM Full "
        "WHERE doi IN ('10.1080/21645515.2017.1330236', '10.1088/0004-6256/135/4/1201')")
cursor.execute(query)
print ('Finish Time: ' + str(datetime.datetime.now()))

In [ ]:
result_set = cursor.fetchall()
for row in result_set:
    print(row)

In [ ]:
sils_doi = ['10.1145/3173574.3173935',
            '10.1108/ILS-06-2018-0049',
            '10.1016/j.lisr.2017.03.001',
            '10.1007/s10803-017-3127-z',
            '10.1145/2854946.2854979',
            '10.1016/j.envsoft.2015.12.010',
            '10.1177%2F2053951715608877',
            '10.2218/ijdc.v10i1.352',
            '10.1145/3209624',
            '10.1016/j.lisr.2017.03.005',
            '10.1109/TVCG.2018.2864885',
            '10.1177%2F016555150202800601',
            '10.1109/TMM.2016.2614220',
            '10.1353/at.2006.0012',
            '10.1007/s10900-018-0547-4',
            '10.1108/JD-03-2016-0030',
            '10.1145/2998181.2998204',
            '10.1145/3209280.3209530',
            '10.1080/10447318.2017.1365459',
            '10.1145/3204461',
            '10.1002/asi.23124',
            '10.1002/asi.23839',
            '10.3389/fncom.2012.00019'
            '10.5210/ojphi.v6i2.5469',
            '10.1093/jamia/ocy013',
            '10.1108/JD-06-2017-0095',
            '10.1016/j.ijinfomgt.2018.07.004']

In [ ]:
str1 = '\',\''.join(sils_doi)
print (str1)

In [ ]:
query = ("SELECT doi, is_oa FROM Snapshot100 "
        "WHERE doi IN (\'{}\')").format(str1)
print (query)
cursor.execute(query)

In [ ]:
print ('Start Time: ' + str(datetime.datetime.now()))
query = ("SELECT doi, is_oa FROM tmp "
        "WHERE doi IN (\'{}\')").format(str1)
cursor.execute(query)
print ('Finish Time: ' + str(datetime.datetime.now()))

In [ ]:
result_set = cursor.fetchall()
for row in result_set:
    print(row)